### Gradient calculation via quantum non demolition measurements

In [146]:
from copy import deepcopy

import numpy as np

from qibo import Circuit, gates, hamiltonians
from qibo.symbols import I, X, Y, Z

In [148]:
def U(
    nqubits: int,
    nlayers: int,
    params: np.ndarray,
    shift_index: int,
    shift_value: float,
    dagger: bool,
    n_ancillas: int = 1,
):
    """Construct 50 shades of circuit."""
    c = Circuit(nqubits + n_ancillas)
    for _ in range(nlayers):
        for q in range(nqubits):
            c.add(gates.RY(q, theta=0.))
            c.add(gates.RZ(q, theta=0.))
        for q in range(nqubits):
            c.add(gates.CNOT(q%nqubits, (q+1)%nqubits))

    tmp_params = deepcopy(params)
    tmp_params[shift_index] += shift_value

    c.set_parameters(tmp_params)

    if dagger:
        return c.invert()
    else:
        return c


def interaction_term(
    hamiltonian: hamiltonians.Hamiltonian, 
    lam: float,
    n_ancillas: int = 1,
    target_ancilla: int = 1,
):
    """Construct interaction between system and detector."""
    tot_qubits = hamiltonian.nqubits + n_ancillas
    target_qubit = hamiltonian.nqubits + target_ancilla - 1
    
    c = Circuit(tot_qubits)

    for q in range(hamiltonian.nqubits):
        c.add(gates.CNOT(q0=q, q1=target_qubit))
    c.add(gates.RZ(target_qubit, theta=lam))
    for q in reversed(range(hamiltonian.nqubits)):
        c.add(gates.CNOT(q0=q, q1=target_qubit))
    return c

def build_external_layer(
    hamiltonian_string: str,
    nqubits: int,
    n_ancillas: int,
    start: bool = True
):
    """Helper function to build final or initial layer."""
    pass                

In [149]:
nqubits = 3
nlayers = 2
n_ancillas = 1
s = np.pi / 2
params = np.random.uniform(-np.pi, np.pi, nqubits * nlayers * 2)

In [150]:
form = 0
for i in range(nqubits):
    form += X(i)

ham = hamiltonians.SymbolicHamiltonian(form)

In [151]:
form

X0 + X1 + X2

In [152]:
u0 = U(
    nqubits=nqubits,
    nlayers=nlayers,
    params=params,
    shift_index=3,
    shift_value=0.,
    dagger=False,
)

u0.draw()

0: ─RY─RZ─o───X─RY─RZ─o───X─
1: ─RY─RZ─X─o─|─RY─RZ─X─o─|─
2: ─RY─RZ───X─o─RY─RZ───X─o─
3: ─────────────────────────


In [153]:
d1 = interaction_term(
    hamiltonian=hamiltonians.SymbolicHamiltonian(form),
    lam=0.01,    
)

d1.draw()

0: ─o────────────o─
1: ─|─o────────o─|─
2: ─|─|─o────o─|─|─
3: ─X─X─X─RZ─X─X─X─


In [154]:
circ = Circuit(nqubits + n_ancillas)
circ.add(gates.H(nqubits))
circ.draw()

0: ───
1: ───
2: ───
3: ─H─


In [155]:
circ += U(
    nqubits=nqubits,
    nlayers=nlayers,
    params=params,
    shift_index=3,
    shift_value=-s,
    dagger=False,
)

In [156]:
circ.draw()

0: ─RY─RZ─o───X─RY─RZ─o───X─
1: ─RY─RZ─X─o─|─RY─RZ─X─o─|─
2: ─RY─RZ───X─o─RY─RZ───X─o─
3: ─H───────────────────────


In [157]:
circ += interaction_term(
    hamiltonian=hamiltonians.SymbolicHamiltonian(form),
    lam=-0.01,  
)

In [158]:
circ += U(
    nqubits=nqubits,
    nlayers=nlayers,
    params=params,
    shift_index=3,
    shift_value=+s,
    dagger=False,
)

In [159]:
circ.draw()

0: ─RY─RZ─o───X─RY─RZ─o───X─o────────────o─RY─RZ─o───X─RY─RZ─o───X─
1: ─RY─RZ─X─o─|─RY─RZ─X─o─|─|─o────────o─|─RY─RZ─X─o─|─RY─RZ─X─o─|─
2: ─RY─RZ───X─o─RY─RZ───X─o─|─|─o────o─|─|─RY─RZ───X─o─RY─RZ───X─o─
3: ─H───────────────────────X─X─X─RZ─X─X─X─────────────────────────


In [160]:
circ += U(
    nqubits=nqubits,
    nlayers=nlayers,
    params=params,
    shift_index=3,
    shift_value=-s,
    dagger=True,
)

In [161]:
circ += interaction_term(
    hamiltonian=hamiltonians.SymbolicHamiltonian(form),
    lam=0.01,  
)

In [162]:
circ

In [163]:
circ.draw()

0:     ─RY─RZ─o───X─RY─RZ─o───X─o────────────o─RY─RZ─o───X─RY─RZ─o───X─X───o─ ...
1:     ─RY─RZ─X─o─|─RY─RZ─X─o─|─|─o────────o─|─RY─RZ─X─o─|─RY─RZ─X─o─|─|─o─X─ ...
2:     ─RY─RZ───X─o─RY─RZ───X─o─|─|─o────o─|─|─RY─RZ───X─o─RY─RZ───X─o─o─X─── ...
3:     ─H───────────────────────X─X─X─RZ─X─X─X─────────────────────────────── ...

0: ... RZ─RY─X───o─RZ─RY─o────────────o─
1: ... RZ─RY─|─o─X─RZ─RY─|─o────────o─|─
2: ... RZ─RY─o─X───RZ─RY─|─|─o────o─|─|─
3: ... ──────────────────X─X─X─RZ─X─X─X─


In [164]:
def final_layer(nqubits, n_ancillas):
    c = Circuit(nqubits+n_ancillas)
    c.add(gates.H(nqubits))
    c.add(gates.S(nqubits))
    c.add(gates.H(nqubits))
    c.add(gates.M(nqubits))
    return c

In [165]:
circ +=final_layer(nqubits, 1)

In [166]:
circ.draw()

0:     ─RY─RZ─o───X─RY─RZ─o───X─o────────────o─RY─RZ─o───X─RY─RZ─o───X─X───o─ ...
1:     ─RY─RZ─X─o─|─RY─RZ─X─o─|─|─o────────o─|─RY─RZ─X─o─|─RY─RZ─X─o─|─|─o─X─ ...
2:     ─RY─RZ───X─o─RY─RZ───X─o─|─|─o────o─|─|─RY─RZ───X─o─RY─RZ───X─o─o─X─── ...
3:     ─H───────────────────────X─X─X─RZ─X─X─X─────────────────────────────── ...

0: ... RZ─RY─X───o─RZ─RY─o────────────o─────────
1: ... RZ─RY─|─o─X─RZ─RY─|─o────────o─|─────────
2: ... RZ─RY─o─X───RZ─RY─|─|─o────o─|─|─────────
3: ... ──────────────────X─X─X─RZ─X─X─X─H─S─H─M─


In [167]:
print(c())

(1+0j)|000>
